In [ ]:
from math import sin, cos, pi      #数学関数・定数のインポート
import numpy as np                 #数値計算機能のインポート 別名np
import matplotlib.pyplot as plt    #グラフ描画機能のインポート 別名plt
import numpy.linalg as la          #線形代数をインポート 別名la
from scipy.integrate import odeint #常微分方程式の解法をインポート
from matplotlib.animation import FuncAnimation #アニメーション機能のインポート
from matplotlib import rc          #各種設定機能のインポート
rc('animation', html='jshtml')     #Colabでアニメーション表示可能にするための設定

# 床面でバウンドする棒

### 運動方程式の定義

#### (1) 符号付き面積(signed area)を計算する関数

In [ ]:
def signed_area(xvec, yvec):

  # ベルトルxvec, yvecを成分とする行列
  ## A = np.array([xvec, yvec]).T #成分の並びをテキストに合わせる場合
  A = np.array([xvec, yvec]) #行列式は転置しても同じなので転置を省略

  # 符号付き面積＝行列式 det(A)
  return la.det(A)

In [ ]:
signed_area([1,2],[3,4]) #お試し

$\begin{bmatrix}1 \\ 2\end{bmatrix} \wedge \begin{bmatrix}3 \\ 4\end{bmatrix}= \begin{vmatrix}1 & 3\\ 2 & 4\end{vmatrix}=1\cdot 4 - 3\cdot2=-2$

#### (2) ステップ関数と符号関数



##### 丸めたステップ関数



In [ ]:
def U(x, s=10000): #s=10000 は丸め具合のデフォルト値

  return 1/(1+np.exp(-s*x)) #式(10.10)　シグモイド関数

In [ ]:
#お試し
xs = np.linspace(-0.5, 0.5, 500) #x軸を表す等差数列
ys50 = U(xs, 50) #対応するy成分．s=50の場合
plt.plot( xs, ys50 )
plt.xlabel('x')
plt.ylabel('U(x)')

##### 丸めた符号関数

In [ ]:
def sgn(x, s=10000): #s=10000 は丸め具合のデフォルト値

  return 2*U(x,s) - 1 #式(11.4)

In [ ]:
#お試し
xs = np.linspace(-0.5, 0.5, 500) #x軸を表す等差数列
ys50 = sgn(xs, 50) #対応するy成分．s=50の場合
plt.plot( xs, ys50 )
plt.xlabel('x')
plt.ylabel('sgn(x)')

#### (3) 床からの反力を計算する関数

In [ ]:
def floor_force(y, dydt, dxdt):

  #垂直抗力（ペナルティー法）
  R = U(-y) * (-Kp*y - Cp*dydt)

  #クーロン摩擦
  F = -sgn(dxdt) * mu * R

  return np.array([F, R])

#### (4) 運動方程式 (equation of motion)

In [ ]:
# パラメータの設定（適当に定めた）
m, l = 1, 1       #質量，棒長
J = m*(l**2)/3    #棒の慣性モーメント
g = 9.8           #重力加速度
mu = 0.3          #摩擦係数（今回は動摩擦係数＝静摩擦係数とみなす）
Kp, Cp = 8000, 25 #床のばね定数，減衰係数 

def EOM(xs, t):

  x, dxdt, y, dydt, Q, dQdt = xs

  # 重心から見た着力点の位置ベクトル
  u1 = -l/2*np.array([cos(Q), sin(Q)])          #式(10.1)
  u2 = - u1

  # 重心から見た着力点の速度ベクトル
  du1dt = l*dQdt/2*np.array([sin(Q), -cos(Q)])  #式(10.13)
  du2dt = - du1dt

  # 床から見た重心の位置ベクトル，速度ベクトル
  X = np.array([x, y])
  dXdt = np.array([dxdt, dydt])

  # 床から見た着力点の位置ベクトル，速度ベクトル
  X1 = X + u1   #1つ目の端点
  dX1dt = dXdt + du1dt
  X2 = X + u2   #2つ目の端点
  dX2dt = dXdt + du2dt

  # 床からの反力（1つ目）
  y1, dy1dt, dx1dt = X1[1], dX1dt[1], dX1dt[0]  #垂直成分，垂直成分，水平成分
  f1 = floor_force(y1, dy1dt, dx1dt) #反力

  # 床からの反力（2つ目）
  y2, dy2dt, dx2dt = X2[1], dX2dt[1], dX2dt[0]  #同上
  f2 = floor_force(y2, dy2dt, dx2dt) #反力

  # 全外力 F と全トルク T
  F = f1 + f2 + np.array([0,-m*g])                #式(10.2)
  T = signed_area(u1, f1) + signed_area(u2, f2)   #式(10.3)
  
  # 1階化した運動方程式の右辺
  dx = np.array([
    dxdt,
    F[0]/m,
    dydt,
    F[1]/m,
    dQdt,
    T/J
  ])

  return dx

### 運動方程式の数値解

#### (5) 時間軸の作成

In [ ]:
t0 = 0    #初期時刻
dt = 0.05 #時間ステップ
tn = 130  #データ長
t1 = t0 + dt*(tn-1) #終端時刻

ts = np.linspace(t0, t1, tn) #時間軸を表す等差数列
ts

#### (6) 数値積分

In [ ]:
x0 = np.array([0, 0, 5, 0, pi/4, 0]) #初期状態

solution = odeint(EOM, x0, ts)      #運動方程式の数値解を求める
solution

#### 時間波形の確認


In [ ]:
fig, axs= plt.subplots(6, 1, figsize=(8,8)) #グラフ用紙(axs)を4行,1列(4枚)用意

for i in range(6):
  ax = axs[i]
  ax.set_xlabel('$t$')
  ax.set_ylabel('$x_%d$' %(i+1))

  xs = solution[:,i]
  ax.plot(ts, xs) 

### 数値解のアニメーション

#### アニメーション用のユーザー関数

In [ ]:
def display_motion(solution):

  #グラフ用紙の設定
  fig, ax= plt.subplots(1, 1, figsize=(8,6)) #グラフ用紙(ax)を1行,1列(1枚)用意

  #アニメーションの１コマの描画
  def each_frame(i):
    ax.cla() #グラフ用紙を白紙にリセット
    ax.set_xlim(-4,4) #x軸の範囲
    ax.set_ylim(0,6) #y軸の範囲
    ax.grid()

    # i番目の数値解
    xs = solution[i,:]

    # 特徴点の座標
    ## 一般化座標の値
    x, y, Q = xs[0], xs[2], xs[4]

    ## 床から見た重心の位置ベクトル，速度ベクトル
    X = np.array([x, y])

    ## 重心から見た着力点の位置ベクトル
    u1 = -l/2*np.array([cos(Q), sin(Q)])          #式(10.1)
    u2 = - u1

    # 床から見た着力点の位置ベクトル
    X1 = X + u1   #1つ目の端点
    X2 = X + u2   #2つ目の端点

    #棒の描画
    ax.plot([X1[0], X2[0]], [X1[1], X2[1]], linewidth=3)

  #アニメーションデータの作成
  anim = FuncAnimation(
      fig, 
      each_frame,
      interval=80,
      frames=len(solution)
  )
  
  return anim

#### (7) 数値解のアニメーション表示

In [ ]:
display_motion(solution)

### 実習3.3
- 実習3.3は，上記の実行例で完了していますが，
- 条件の変更に必要な部分だけ集めて，再実行しておきます．実行結果は上記と同じです．

In [ ]:
#### パラメータの設定（適当に定めた）
m, l = 1, 1       #質量，棒長
J = m*(l**2)/3    #棒の慣性モーメント
g = 9.8           #重力加速度
mu = 0.3          #摩擦係数（今回は動摩擦係数＝静摩擦係数とみなす）
Kp, Cp = 8000, 25 #床のばね定数，減衰係数 

### (1)〜(4) ※共通なので再掲せず

### (5) 時間軸の作成
t0 = 0    #初期時刻
dt = 0.05 #時間ステップ
tn = 130  #データ長
t1 = t0 + dt*(tn-1) #終端時刻
ts = np.linspace(t0, t1, tn) #時間軸を表す等差数列

### (6) 数値積分
x0 = np.array([0, 0, 5, 0, pi/4, 0])  #初期状態
solution = odeint(EOM, x0, ts)        #運動方程式の数値解を求める

### (7) 数値解のアニメーション表示
display_motion(solution)